In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import csv

In [2]:
def parse_data(filename, label) :
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
#     img = tf.image.resize(img_decode, [227,227])
    return img_decode, label

def create_dataset(filenames, labels, batch_size) :
    dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
    dataset = dataset.map(parse_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

def read_labels(path) :
    img_classes = os.listdir(path)
    total_len = len(os.listdir(path+img_classes[0])) + len(os.listdir(path+img_classes[1])) + \
    len(os.listdir(path+img_classes[2]))+ len(os.listdir(path+img_classes[3])) + \
    len(os.listdir(path+img_classes[4]))+ len(os.listdir(path+img_classes[5]))+\
    len(os.listdir(path+img_classes[6]))

    names = []
    labels = np.zeros((total_len, len(img_classes)))
    j=0
    for i, img_class in enumerate(img_classes) :
        for img in os.listdir(os.path.join(path, img_class)) :
            names.append(os.path.join(path, img_class, img))
            labels[j][i] = 1
            j+=1
        
    return names, labels


batch_size = 16
images, labels = read_labels('./train/')
dataset = create_dataset(images, labels, batch_size)

In [7]:
from keras import Sequential, layers, Model

resnet50 = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(227,227,3))

for i, layer in enumerate(resnet50.layers) :
    if i < 103 :
        layer.trainable = False
        
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 233, 233, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 114, 114, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 114, 114, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [8]:
model = Sequential()
model.add(resnet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 19,465,223
Non-trainable params: 4,136,832
_________________________________________________________________


In [9]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)

In [11]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=4)

Epoch 1/4
107/107 [==============================] - 371s 3s/step - loss: 1.1619 - accuracy: 0.6567 - precision: 0.7508 - recall: 0.5395 - AUC: 0.9044
Epoch 2/4
107/107 [==============================] - 347s 3s/step - loss: 0.4715 - accuracy: 0.8475 - precision: 0.8867 - recall: 0.7974 - AUC: 0.9801
Epoch 3/4
107/107 [==============================] - 367s 3s/step - loss: 0.2164 - accuracy: 0.9299 - precision: 0.9479 - recall: 0.9099 - AUC: 0.9957
Epoch 4/4
107/107 [==============================] - 351s 3s/step - loss: 0.1415 - accuracy: 0.9547 - precision: 0.9684 - recall: 0.9399 - AUC: 0.9975


In [13]:
from tqdm import tqdm

test_path = './test/0'

with open('test_answer2.csv', 'w' , newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for i, image in tqdm(enumerate(os.listdir(test_path))) :
        img = cv2.imread(os.path.join(test_path, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(227,227))
        img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))

        pred = model.predict(img)
        answer = np.argmax(pred[0])
        wr.writerow([i, answer])
    

350it [00:40,  8.68it/s]
